### Deploy Web App on Azure Container Services (AKS)
In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored.
    
The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Connect to AKS](#section3)
* [Deploy our app](#section5)


This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

<a id='section1'></a>
## Setup
Below are the various name definitions for the resources needed to setup ACS as well as the name of the Docker image we will be using.

**Some outputs (and inputs) below have been hidden/masked for confidentiality**

In [1]:
# Please modify the below as you see fit
resource_group = "ml-aks" 
aks_name = "demoaks"
location = "eastus"

image_name = 'lenisha/tfresnet-gpu' # 'masalvar/tfresnet-gpu' Feel free to use this Image if you want to 
                                   # skip creating your own container
selected_subscription = "'f869415f-5cff-46a3-b728-20659d14d62d'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

<a id='section2'></a>
## Azure account login
The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [2]:

!az login --service-principal -u http://vstspacker -p vstspacker123! --tenant 72f988bf-86f1-41af-91ab-2d7cd011db47
!az account show

[
  {
    "cloudName": "AzureCloud",
    "id": "f869415f-5cff-46a3-b728-20659d14d62d",
    "isDefault": true,
    "name": "Microsoft Azure Internal Consumption",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "http://vstspacker",
      "type": "servicePrincipal"
    }
  }
]
{
  "environmentName": "AzureCloud",
  "id": "f869415f-5cff-46a3-b728-20659d14d62d",
  "isDefault": true,
  "name": "Microsoft Azure Internal Consumption",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "http://vstspacker",
    "type": "servicePrincipal"
  }
}


### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [3]:
!az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


<a id='section4'></a>
## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [4]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "demoaks" as current context in /home/mmlspark/.kube/config


Let's verify connection by listing the nodes.

In [5]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-agentpool-23589211-0   Ready     agent     16h       v1.9.6
aks-agentpool-23589211-1   Ready     agent     1h        v1.9.6
aks-agentpool-23589211-2   Ready     agent     1h        v1.9.6


Let's check the pods on our cluster.

In [6]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                                              READY     STATUS             RESTARTS   AGE
default       azure-dl-f568985df-4hjr9                                          1/1       Running            0          23m
kube-system   addon-http-application-routing-default-http-backend-66c97ffn2hl   1/1       Running            0          16h
kube-system   addon-http-application-routing-external-dns-8d5bf7cd7-jkmmf       1/1       Running            0          16h
kube-system   addon-http-application-routing-nginx-ingress-controller-64888pj   1/1       Running            0          16h
kube-system   heapster-7cd8dd888b-rx8sl                                         1/2       CrashLoopBackOff   194        16h
kube-system   kube-dns-v20-7c556f89c5-bxv6z                                     3/3       Running            0          16h
kube-system   kube-dns-v20-7c556f89c5-hbbpr                                     3/3       Running            0          16h
kube-sys

<a id='section5'></a>
## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.


In [7]:
app_template = {
    "apiVersion": "apps/v1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "selector": {
          "matchLabels": {"app": "azure-dl"}
          
        },
        "strategy": {
          "type": "RollingUpdate",
          "rollingUpdate": {
            "maxSurge": 1,
            "maxUnavailable": 1
          }
        },
        "minReadySeconds": 15,

        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                            }
                        ],
                        "image": image_name,
                        
                        "name": "azure-dl",
                       
                        
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "alpha.kubernetes.io/nvidia-gpu": 1
                            },
                            "requests": {
                                "alpha.kubernetes.io/nvidia-gpu": 1
                            }
                        },
                        "volumeMounts": [
                            {
                                "mountPath": "/usr/local/nvidia",
                                "name": "nvidia"
                            }
                        ]
                    }
                ],
                "volumes": [
                    {
                        "hostPath": {
                            "path": "/usr/local/nvidia"
                        },
                        "name": "nvidia"
                    }
                ]
            }
        }
    }
}




service_temp = {
  
    "apiVersion": "v1",
    "kind": "Service",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "ports": [
            {
                "port": 80
            }
        ],
        "selector": {
            "app": "azure-dl"
        },
        "type": "LoadBalancer"
    }
}

In [8]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4,sort_keys=True)
        outfile.write('\n\n')

In [9]:
write_json_to_file(app_template, 'az-dl.json') # We write the service template to the json file

In [10]:
write_json_to_file(service_temp, 'az-dl.json', mode='a') # We add the loadbelanacer template to the json file

Let's check the manifest created.

In [11]:
!cat az-dl.json

{
    "apiVersion": "apps/v1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "minReadySeconds": 15,
        "replicas": 1,
        "selector": {
            "matchLabels": {
                "app": "azure-dl"
            }
        },
        "strategy": {
            "rollingUpdate": {
                "maxSurge": 1,
                "maxUnavailable": 1
            },
            "type": "RollingUpdate"
        },
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                            }
                        ],
                   

Next, we will use kubectl create command to deploy our application.

In [ ]:
!kubectl apply -f az-dl.json

Let's check if the pod is deployed.

In [19]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                                              READY     STATUS             RESTARTS   AGE
default       azure-dl-5859fc8467-hk9gx                                         1/1       Running            0          1h
default       azure-dl-5859fc8467-pcxt5                                         1/1       Running            0          35s
default       azure-dl-5859fc8467-sgdbn                                         1/1       Running            0          35s
kube-system   addon-http-application-routing-default-http-backend-66c97ffn2hl   1/1       Running            0          15h
kube-system   addon-http-application-routing-external-dns-8d5bf7cd7-jkmmf       1/1       Running            0          15h
kube-system   addon-http-application-routing-nginx-ingress-controller-64888pj   1/1       Running            0          15h
kube-system   heapster-7cd8dd888b-rx8sl                                         1/2       CrashLoopBackOff   183        15h
kube-syst

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [20]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND                      SUBOBJECT                   TYPE      REASON                         SOURCE                                 MESSAGE
50m         50m          1         aks-agentpool-23589211-1.15329f359402b61b    Node                                                  Normal    Starting                       kubelet, aks-agentpool-23589211-1      Starting kubelet.
50m         50m          2         aks-agentpool-23589211-1.15329f35986ec240    Node                                                  Normal    NodeHasSufficientDisk          kubelet, aks-agentpool-23589211-1      Node aks-agentpool-23589211-1 status is now: NodeHasSufficientDisk
50m         50m          2         aks-agentpool-23589211-1.15329f35986ef7c4    Node                                                  Normal    NodeHasSufficientMemory        kubelet, aks-agentpool-23589211-1      Node aks-agentpool-23589211-1 status is now: NodeHa

48m         48m          1         azure-dl-5859fc8467-sj22w.15329f5822f9107d   Pod                                                   Normal    SandboxChanged                 kubelet, aks-agentpool-23589211-1      Pod sandbox changed, it will be killed and re-created.
44m         44m          1         azure-dl-5859fc8467-sj22w.15329f87b850191f   Pod                       spec.containers{azure-dl}   Normal    Pulled                         kubelet, aks-agentpool-23589211-1      Successfully pulled image "mldemoacr.azurecr.io/ml/tfresnet-gpu"
44m         44m          1         azure-dl-5859fc8467-sj22w.15329f87c4fb8697   Pod                       spec.containers{azure-dl}   Normal    Created                        kubelet, aks-agentpool-23589211-1      Created container
44m         44m          1         azure-dl-5859fc8467-sj22w.15329f87ccb9b575   Pod                       spec.containers{azure-dl}   Normal    Started                        kubelet, aks-agentpool-23589211-1      Starte

In [21]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-05-27 21:37:04,421 CRIT Supervisor running as root (no user in config file)
2018-05-27 21:37:04,423 INFO supervisord started with pid 1
2018-05-27 21:37:05,425 INFO spawned: 'program_exit' with pid 10
2018-05-27 21:37:05,427 INFO spawned: 'nginx' with pid 11
2018-05-27 21:37:05,428 INFO spawned: 'gunicorn' with pid 12
2018-05-27 21:37:06,460 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-05-27 21:37:07.000605: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-05-27 21:37:07.194163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: c0fb:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-05-27 21:37:07.194207: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1120] Creating TensorFlow de

It can take a few minutes for the service to populate the EXTERNAL-IP field. This will be the IP you use to call the service. You can also specify an IP to use please see the AKS documentation for further details.

In [22]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP   EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.1.67    104.211.53.76   80:32186/TCP   1h


Now that we have our deployed service we can move onto [testing it](02_TestWebApp.ipynb)  
Below are the instructions to tear everything down once we are done with the cluster